# Alpaca Demo

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

%matplotlib inline

In [2]:
# Load .env environment variables
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [5]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [6]:
# Format current date as ISO format
today = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

In [7]:
# Set the tickers
tickers = ["AMC", "GME"]

In [8]:
# Set timeframe to one day ("1Day") for the Alpaca API
timeframe = "1Day"

In [9]:
# Get current closing prices for FB and TWTR
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = today,
    end = today
).df

# Display sample data
df_portfolio

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2020-07-14 04:00:00+00:00,4.25,4.27,4.1000,4.22,3291433,17278,4.184732,AMC
2020-07-14 04:00:00+00:00,4.22,4.31,4.0725,4.08,2261589,12777,4.148324,GME


In [10]:
# Format start and end dates as ISO format for one year period
start = pd.Timestamp("2019-07-14", tz="America/New_York").isoformat()
end = pd.Timestamp("2020-07-14", tz="America/New_York").isoformat()

In [11]:
# Get closing prices for FB and TWTR from the last year
df_portfolio_year = alpaca.get_bars(
    tickers,
    timeframe,
    start = start,
    end = end
).df

# Display sample data
df_portfolio_year.head(10)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2019-07-15 04:00:00+00:00,9.41,9.53,9.06,9.49,2266464,11229,9.344699,AMC
2019-07-16 04:00:00+00:00,9.39,9.82,9.32,9.78,2189561,12855,9.639196,AMC
2019-07-17 04:00:00+00:00,9.78,9.94,9.64,9.81,1753319,10759,9.781631,AMC
2019-07-18 04:00:00+00:00,9.62,9.90,9.49,9.62,2066858,12816,9.648325,AMC
2019-07-19 04:00:00+00:00,9.95,11.04,9.93,10.54,11043544,39237,10.549882,AMC
2019-07-22 04:00:00+00:00,10.64,10.90,10.07,10.41,5519681,28116,10.361358,AMC
2019-07-23 04:00:00+00:00,10.26,10.53,10.18,10.30,2987154,17820,10.310982,AMC
2019-07-24 04:00:00+00:00,10.33,10.89,10.33,10.78,2661289,14687,10.704035,AMC
2019-07-25 04:00:00+00:00,10.81,11.02,10.67,10.94,2211448,12878,10.845264,AMC


In [12]:
# Reorganize the DataFrame

# Separate ticker data
FB = df_portfolio_year[df_portfolio_year['symbol']=='FB'].drop('symbol', axis=1)
TWTR = df_portfolio_year[df_portfolio_year['symbol']=='TWTR'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
df_portfolio_year = pd.concat([FB, TWTR],axis=1, keys=['FB','TWTR'])

# Display sample data
df_portfolio_year

Empty DataFrame
Columns: [(FB, open), (FB, high), (FB, low), (FB, close), (FB, volume), (FB, trade_count), (FB, vwap), (TWTR, open), (TWTR, high), (TWTR, low), (TWTR, close), (TWTR, volume), (TWTR, trade_count), (TWTR, vwap)]
Index: []

In [13]:
df_portfolio_year.tail(10)

Empty DataFrame
Columns: [(FB, open), (FB, high), (FB, low), (FB, close), (FB, volume), (FB, trade_count), (FB, vwap), (TWTR, open), (TWTR, high), (TWTR, low), (TWTR, close), (TWTR, volume), (TWTR, trade_count), (TWTR, vwap)]
Index: []

In [14]:
# Create and empty DataFrame for closing prices
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of FB and TWTR
df_closing_prices["FB"] = df_portfolio_year["FB"]["close"]
df_closing_prices["TWTR"] = df_portfolio_year["TWTR"]["close"]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

# Display sample data
df_closing_prices.head(10)

AttributeError: 'Index' object has no attribute 'date'

In [ ]:
# Compute daily returns
df_daily_returns = df_closing_prices.pct_change().dropna()

# Display sample data
df_daily_returns.head()

In [ ]:
# Plot daily returns
df_daily_returns.plot(title="Daily Returns of FB and TWTR over the Last Year")